In [1]:
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

DATA_SET = "dataset/output"

image_size = 160
batch_size = 32
classes = 6
epochs = 5
lr = 0.0001

In [2]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)  # set validation split

# print("type of {}".format(type(train_datagen)))

train_generator = train_datagen.flow_from_directory(
    DATA_SET,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    subset='training')  # set as training data

# print("type of {}".format(type(train_generator)))

validation_generator = train_datagen.flow_from_directory(
    DATA_SET,  # same directory as training data
    target_size=(image_size, image_size),
    batch_size=batch_size,
    subset='validation')  # set as validation data
# print("type of {}".format(type(validation_generator)))

Found 141 images belonging to 6 classes.
Found 33 images belonging to 6 classes.


In [3]:
base = tf.keras.applications.MobileNetV2(
    include_top=False,
    alpha=0.35,
    weights='imagenet',
    input_tensor=Input(shape=(image_size, image_size, 3)),
    input_shape=(image_size, image_size, 3))

base.trainable = False

base_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet',
                                               input_tensor=Input(shape=(image_size, image_size, 3)),
                                               input_shape=(image_size, image_size, 3))
top_layers = base_model.output
top_layers = GlobalAveragePooling2D()(top_layers)
top_layers = Dense(1024, activation='relu')(top_layers)
predictions = Dense(classes, activation='softmax')(top_layers)
model = Model(inputs=base_model.input, outputs=predictions)

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][0]']           

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

model.save("save_model.h5")

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()